<a href="https://colab.research.google.com/github/giozu/FEniCS/blob/main/2D%20problems/2d_poisson_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Equation and problem definition
Given a domain $\Omega \subset \mathbb{R}^n$ with boundary $\partial
\Omega = \Gamma_{D} \cup \Gamma_{N}$, the Poisson equation with
particular boundary conditions is:

$$
\begin{align}
- D \nabla^{2} u &= S \quad {\rm in} \ \Omega, \\
u &= 0 \quad {\rm on} \ \partial \Omega, \\
\end{align}
$$

In this particular case, we consider $u$ to be a concentration,  $S$ a production rate, and $D$ a diffusivity. The variational problem reads: find $u \in V$ such
that

$$
a(u, v) = L(v) \quad \forall \ v \in V,
$$

where $V$ is a suitable function space and

$$
\begin{align}
a(u, v) &:= \int_{\Omega} \nabla u \cdot \nabla v \, {\rm d} x, \\
L(v)    &:= \int_{\Omega} S v \, {\rm d} x
\end{align}
$$

The expression $a(u, v)$ is a bilinear form and $L(v)$ is a linear functional. It is assumed that all functions in $V$ satisfy the boundary conditions.

## Initializing notebook plotting
When running a python notebook, a special line is necessary to show the plots.

In [1]:
%matplotlib inline

## Implementation
The modules that will be used are imported:

In [2]:
import numpy as np

import ufl
from ufl import ds, dx, grad, inner

from dolfinx import fem, io, mesh, plot
from dolfinx.io import gmshio # to import gmsh functions
from dolfinx import geometry # to define the line plot

from mpi4py import MPI

# from petsc4py import PETSc
from petsc4py.PETSc import ScalarType

import matplotlib.pyplot as plt

### Mesh generation and functional spaces
We create the mesh using dedicated functions (e.g., {py:func} `create_rectangle<dolfinx.mesh.create_rectangle>`), or we import it from a .msh file (made with gmsh). Then, we define a finite element space $V$ on the mesh. In this case `V` is the space of continuous Lagrange finite elements of degree 1.

To apply Dirichlet boundary conditions, we find the mesh facets (entities of topological co-dimension 1) that lie on the boundary $\Gamma_D = \partial \Omega$ using `locate_entities_boundary`. The function is provided with a boolean function that returns `True` for points `x` on the boundary and `False` otherwise.

Lastly, we find the degrees-of-freedom that are associated with the boundary facets using `locate_dofs_topological`, and use `dirichletbc` to create a `DirichletBCMetaClass` class that represents the boundary condition.

In [3]:
# Mesh
# Rectangle
# msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
#                             points=((0.0, 0.0), (2.0, 1.0)), n=(32, 16),
#                             cell_type=mesh.CellType.triangle,)
# Square
# msh = mesh.create_unit_square(MPI.COMM_WORLD, 12, 12, mesh.CellType.quadrilateral)

# Hexagon (from .msh file, generated with gmsh)
# msh, _, _ = gmshio.read_from_msh("esagonino.msh", MPI.COMM_WORLD, gdim = 2)
msh, cell_tags, facet_tags = gmshio.read_from_msh("esagonino.msh", MPI.COMM_WORLD, gdim = 2)

# Boundary conditions
# Create facet to cell connectivity required to determine boundary facets
tdim = msh.topology.dim
fdim = tdim - 1
msh.topology.create_connectivity(fdim, tdim)

# Finite element space, defined on the mesh
# The second argument of `FunctionSpace` is a tuple `(family, degree)`,
# where `family` is the finite element family, and `degree` specifies the polynomial degree.
V = fem.FunctionSpace(msh, ("Lagrange", 1))

# facets = mesh.locate_entities_boundary(msh, dim=(msh.topology.dim - 1),
#                                        marker=lambda x: np.logical_or(np.isclose(x[0], 0.0),
#                                                                       np.isclose(x[0], 2.0)))
facets = mesh.exterior_facet_indices(msh.topology)

# Degrees of freedom
dofs = fem.locate_dofs_topological(V=V, entity_dim=fdim, entities=facets)
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)

# f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
S = 3 # production rate (1e-7)
# g = ufl.sin(5 * x[0])
D = 0.3 # diffusivity

a = inner(grad(u), grad(v)) * dx
L = inner(S/D, v) * dx # + inner(g, v) * ds

# Variational problem
# The `LinearProblem` object brings together the variational problem, the Dirichlet boundary condition, and the linear solver.
# In this case an LU solver is sued. The function `solve` computes the solution.
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

# The solution can be written to a file, and visualize it with ParaView or VisIt:
# with io.XDMFFile(msh.comm, "out_poisson/poisson.xdmf", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(uh)

Info    : Reading 'esagonino.msh'...
Info    : 13 entities
Info    : 817 nodes
Info    : 1536 elements
Info    : Done reading 'esagonino.msh'


### Class definition

In [ ]:
class poisson_problem():
    def __init__(self):
        
        # Domain
        self.domain, __, __ = gmshio.read_from_msh("esagonino.msh", MPI.COMM_WORLD, gdim = 2)
        self.tdim = self.domain.topology.dim
        self.fdim = self.tdim - 1
        self.domain.topology.create_connectivity(self.fdim, self.tdim)

        # Functional spaces
        self.P1 = ufl.FiniteElement("Lagrange", self.domain.ufl_cell(), 1)
        self.V = fem.FunctionSpace(self.domain, self.P1)

        # Test and trial functions
        self.u = ufl.TestFunction(self.V)
        self.v = ufl.TrialFunction(self.V)
        self.x = ufl.SpatialCoordinate(self.domain)

    def parameters(self, f):
        self.forcingValue = f
        print('forcing = {:.0f}'.format(f))
        self.forcing = fem.Constant(self.domain, ScalarType(self.forcingValue))

    def boundaryConditions(self):
        self.facets = mesh.exterior_facet_indices(self.domain.topology)
        self.dofs = fem.locate_dofs_topological(V=self.V, entity_dim=self.fdim, entities=self.facets)
        self.bc = fem.dirichletbc(value=self.forcing, dofs=self.dofs, V=self.V)

    def assemble(self):
        # Variational forms
        self.lhs = inner(grad(self.u), grad(self.v)) * dx
        self.rhs = inner(self.forcingValue, self.v) * dx

    def solve(self):
        self.problem = fem.petsc.LinearProblem(self.lhs, self.lhs, bcs=[self.bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
        self.solution = self.problem.solve()
        return self.solution

Problem solution

In [5]:
poisson = poisson_problem()

S = 3 # production rate (1e-7)
D = 0.3 # diffusivity
forcing = S / D

poisson.parameters(forcing)
poisson.boundaryConditions()
poisson.assemble()
uh = poisson.solve()

: 

: 

## Visualization
displayed using [pyvista](https://docs.pyvista.org/).


In [ ]:
try:
    import pyvista
    cells, types, x = plot.create_vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = uh.x.array.real
    grid.set_active_scalars("u")
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, show_edges=True)
    warped = grid.warp_by_scalar()
    plotter.add_mesh(warped)
    if pyvista.OFF_SCREEN:
        pyvista.start_xvfb(wait=0.1)
        plotter.screenshot("uh_poisson.png")
    else:
        plotter.show()

except ModuleNotFoundError:
    print("'pyvista' is required to visualise the solution")
    print("Install 'pyvista' with pip: 'python3 -m pip install pyvista'")


Widget(value="<iframe src='http://localhost:35913/index.html?ui=P_0x7feadf7a1ae0_0&reconnect=auto' style='widt…

## Curve plots throughout the mesh
It is possible to plot the finite element solution $u_h$ along a determined line $y=f(x)$. First, we define the set of points belonging to the line, then we evaluate $u_h$ at these points.

Since the finite element solution is a linear combination of all degrees of freedom, $u_h = \sum_{i=1}^N c_i \phi_i(x)$, where $c_i$ is the coefficient of $u_i$ and $\phi_i$ is the basis function, we can compute the solution at any point in $\Omega$. However, as a mesh consists of a large set of degrees of freedom (i.e. $N$ is large), we want to reduce the number of evaluations of the basis functions. This is efficiently done by creating a bounding box tree of the cells of the mesh, allowing a quick recursive search through the mesh entities.

In [ ]:
points = np.zeros((3, 101))

# plot along the line y = x
points[0] = np.linspace(0, 1, 101)
points[1] = np.linspace(0, 1, 101)

u_values = []
cells = []
points_on_proc = []

bb_tree = geometry.BoundingBoxTree(msh, msh.topology.dim)

cell_candidates = geometry.compute_collisions(bb_tree, points.T)
colliding_cells = geometry.compute_colliding_cells(msh, cell_candidates, points.T)

for i, point in enumerate(points.T):
    if len(colliding_cells.links(i))>0:
        points_on_proc.append(point)
        cells.append(colliding_cells.links(i)[0])
        
points_on_proc = np.array(points_on_proc, dtype=np.float64)
u_values = uh.eval(points_on_proc, cells)

In [ ]:
# 2D plot
u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)

u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

# Line plot
fig = plt.figure()
plt.plot(points_on_proc[:,1], u_values, "k", linewidth=2, label="Deflection ($\\times 50$)")
plt.grid(True)
plt.xlabel("y")
plt.legend()

NameError: name 'pyvista' is not defined

## Evaluate the gradient of the solution
We evaluate the gradient of $u_h$ by defining an expression for the gradient and interpolating it on the domain.

In [ ]:
P2 = ufl.VectorElement("Lagrange", msh.ufl_cell(), 1)
W = fem.FunctionSpace(msh, P2)

# grad_u_expr = fem.Expression(grad(uh)[0], V.element.interpolation_points()) # d/dx
grad_u_expr = fem.Expression(grad(uh), W.element.interpolation_points())
grad_u = fem.Function(W)
grad_u.interpolate(grad_u_expr)

# 2D plot
u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)

u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = grad_u.sub(0).collapse().x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

# The solution can be written to a file, and visualize it with ParaView or VisIt:
with io.XDMFFile(msh.comm, "out_poisson/poisson.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(grad_u)


Widget(value="<iframe src='http://localhost:43769/index.html?ui=P_0x7f485c5be350_8&reconnect=auto' style='widt…